In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install rouge_score
!pip install contractions
!pip install accelerate
!pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
import os
import re
import contractions
import torch
import nltk
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BartForConditionalGeneration, BartTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW
from rouge_score import rouge_scorer
from accelerate import Accelerator
from nltk.corpus import stopwords

In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer
import re
import contractions
from nltk.corpus import stopwords
import nltk

# Download stopwords
nltk.download('stopwords')

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
# Display the first few entries of the dataset
print("Train dataset sample:")
for i in range(3):
    print(f"Article {i+1}:\n{dataset['train'][i]['article']}\n")
    print(f"Summary {i+1}:\n{dataset['train'][i]['highlights']}\n")
    print("--------------------------------------------------------\n")


Train dataset sample:
Article 1:
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK

In [ ]:
# Summarize the lengths of articles and summaries
def calculate_length_stats(data, field):
    lengths = [len(item.split()) for item in data[field]]  # Calculate the length of each article/summary in words
    avg_length = sum(lengths) / len(lengths)  # Average length
    max_length = max(lengths)  # Maximum length
    return avg_length, max_length

avg_article_length, max_article_length = calculate_length_stats(dataset['train'], 'article')
avg_summary_length, max_summary_length = calculate_length_stats(dataset['train'], 'highlights')

print(f"Average article length: {avg_article_length}")
print(f"Max article length: {max_article_length}")
print(f"Average summary length: {avg_summary_length}")
print(f"Max summary length: {max_summary_length}")

Average article length: 691.8703263175126
Max article length: 2347
Average summary length: 51.574101486174435
Max summary length: 1296


In [ ]:
import torch
import os

# Function to clean and preprocess text
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    return text

# Function to preprocess the dataset
def preprocess_data(data):
    inputs = [clean_text(article) for article in data['article']]
    targets = [clean_text(highlight, remove_stopwords=False) for highlight in data['highlights']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=150, truncation=True, padding='max_length', return_tensors='pt')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Initialize the tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Preprocess the data
#train_data = preprocess_data(dataset['train'])
#valid_data = preprocess_data(dataset['validation'])
#test_data = preprocess_data(dataset['test'])

# Save preprocessed data
#os.makedirs('/content/drive/MyDrive/Colab Notebooks/preprocessed_data', exist_ok=True)
#torch.save(train_data, '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/train_data.pt')
#torch.save(valid_data, '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/valid_data.pt')
#torch.save(test_data, '/content/drive/MyDrive/Colab Notebooks/preprocessed_data/test_data.pt')


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

# Load preprocessed data
train_data = torch.load('/content/drive/MyDrive/Colab Notebooks/preprocessed_data/train_data.pt')
valid_data = torch.load('/content/drive/MyDrive/Colab Notebooks/preprocessed_data/valid_data.pt')
test_data = torch.load('/content/drive/MyDrive/Colab Notebooks/preprocessed_data/test_data.pt')

# Create TensorDatasets
train_dataset = TensorDataset(train_data['input_ids'], train_data['attention_mask'], train_data['labels'])
valid_dataset = TensorDataset(valid_data['input_ids'], valid_data['attention_mask'], valid_data['labels'])
test_dataset = TensorDataset(test_data['input_ids'], test_data['attention_mask'], test_data['labels'])


In [ ]:
from torch.utils.data import random_split

# Use a subset of the data
subset_size = 5000  # Adjust this number as needed
train_subset, _ = random_split(train_dataset, [subset_size, len(train_dataset) - subset_size])
valid_subset, _ = random_split(valid_dataset, [1000, len(valid_dataset) - 1000])
test_subset, _ = random_split(test_dataset, [1000, len(test_dataset) - 1000])

# Create DataLoaders with a batch size of 16
train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_subset, batch_size=16)
test_loader = DataLoader(test_subset, batch_size=16)


In [ ]:
def save_checkpoint(epoch, model, optimizer, scheduler, accelerator, path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': accelerator.unwrap_model(model).state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }
    torch.save(checkpoint, path)

def load_checkpoint(path, model, optimizer, scheduler, accelerator):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint['epoch']


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import AdamW
from rouge_score import rouge_scorer
from accelerate import Accelerator
import torch
import os
import time
import re
import contractions
from nltk.corpus import stopwords
import nltk


In [ ]:
from transformers import BartForConditionalGeneration, get_linear_schedule_with_warmup
from accelerate import Accelerator
import torch
import os
import time

# Initialize the accelerator for mixed precision training
accelerator = Accelerator(mixed_precision="fp16")

# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Learning rate scheduler
epochs = 1  # Set to 1 for initial testing
num_training_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Prepare everything with the accelerator
model, optimizer, scheduler = accelerator.prepare(model, optimizer, scheduler)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

NameError: name 'train_loader' is not defined

In [ ]:
# Checkpoint directory
checkpoint_dir = "/content/drive/MyDrive/Colab Notebooks/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Load checkpoint if exists
last_checkpoint = None
start_epoch = 0
for epoch in range(epochs, 0, -1):
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    if os.path.exists(checkpoint_path):
        last_checkpoint = checkpoint_path
        break

if last_checkpoint:
    print(f"Loading checkpoint from {last_checkpoint}")
    start_epoch = load_checkpoint(last_checkpoint, model, optimizer, scheduler, accelerator)


In [ ]:
from rouge_score import rouge_scorer

# Training loop with validation metrics
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
best_val_loss = float('inf')

# Set gradient accumulation steps
accumulation_steps = 2  # Adjust as needed to simulate larger batch size

for epoch in range(start_epoch, epochs):
    start_time = time.time()  # Start time for the epoch
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    for step, batch in enumerate(train_loader):
        input_ids, attention_mask, labels = [t.to(device) for t in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / accumulation_steps
        accelerator.backward(loss)

        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        total_loss += loss.item()

        # Print progress for every 100 steps
        if step % 100 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Training Loss: {loss.item()}")

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Average Training Loss: {avg_train_loss}")

    # Validation
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in valid_loader:
            input_ids, attention_mask, labels = [t.to(device) for t in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
            preds = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            all_preds.extend(preds)
            all_labels.extend(labels)
    avg_val_loss = total_val_loss / len(valid_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss}")

    # Calculate ROUGE scores
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in all_preds]
    labels = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in all_labels]
    rouge1, rougeL = 0, 0
    for pred, label in zip(preds, labels):
        score = scorer.score(pred, label)
        rouge1 += score['rouge1'].fmeasure
        rougeL += score['rougeL'].fmeasure
    rouge1 /= len(preds)
    rougeL /= len(preds)
    print(f"Epoch {epoch+1}, ROUGE-1: {rouge1}, ROUGE-L: {rougeL}")

    # Save the best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        accelerator.save_state("best_model")

    # Save checkpoint after every epoch
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}.pth")
    save_checkpoint(epoch + 1, model, optimizer, scheduler, accelerator, checkpoint_path)

    end_time = time.time()  # End time for the epoch
    epoch_duration = end_time - start_time
    print(f"Epoch {epoch+1} completed in {epoch_duration:.2f} seconds.")


Epoch 1, Step 0, Training Loss: 0.3922291100025177
Epoch 1, Step 100, Training Loss: 0.33129647374153137
Epoch 1, Step 200, Training Loss: 0.3842114508152008
Epoch 1, Step 300, Training Loss: 0.34759634733200073
Epoch 1, Average Training Loss: 0.36080404668570326
Epoch 1, Validation Loss: 0.9287444447714185


Epoch 1, ROUGE-1: 0.3964794141836771, ROUGE-L: 0.2767976005727075
Epoch 1 completed in 575.59 seconds.


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained(model_dir)
tokenizer = BartTokenizer.from_pretrained(model_dir)
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_l

In [ ]:
def generate_summary(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True).to(device)
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# Test the model with a sample input
sample_input = "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving. The ideal characteristic of artificial intelligence is its ability to rationalize and take actions that have the best chance of achieving a specific goal. AI is being used across different industries to automate processes, analyze large sets of data, and enhance customer experiences. Despite its potential benefits, AI also poses significant ethical concerns, including issues related to privacy, job displacement, and decision-making transparency."
original_summary = "Artificial Intelligence (AI) simulates human intelligence in machines capable of performing tasks that typically require human intelligence, such as learning and problem-solving. AI is widely used to automate processes, analyze data, and improve customer experiences, but it also raises ethical concerns about privacy and job displacement."
print("Original summary:", original_summary)
print("Generated summary:", generate_summary(sample_input))


Original summary: Artificial Intelligence (AI) simulates human intelligence in machines capable of performing tasks that typically require human intelligence, such as learning and problem-solving. AI is widely used to automate processes, analyze data, and improve customer experiences, but it also raises ethical concerns about privacy and job displacement.
Generated summary: artificial intelligence is the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions  
it is being used across different industries to automate processes and analyze large sets of data and enhance customer experiences  


In [ ]:
# Use a subset of the test data
test_subset_size = 10  # Adjust this number as needed
test_subset, _ = random_split(test_dataset, [test_subset_size, len(test_dataset) - test_subset_size])

# Create DataLoader for the subset
test_loader = DataLoader(test_subset, batch_size=16)


In [ ]:
from rouge_score import rouge_scorer

def evaluate_model(test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [t.to(device) for t in batch]
            preds = model.generate(input_ids.to(device), max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            all_preds.extend(preds)
            all_labels.extend(labels)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in all_preds]
    labels = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in all_labels]
    rouge1, rougeL = 0, 0
    for pred, label in zip(preds, labels):
        score = scorer.score(pred, label)
        rouge1 += score['rouge1'].fmeasure
        rougeL += score['rougeL'].fmeasure
    rouge1 /= len(preds)
    rougeL /= len(preds)
    print(f"ROUGE-1: {rouge1}, ROUGE-L: {rougeL}")

# Evaluate the model on the subset
evaluate_model(test_loader)


ROUGE-1: 0.3487559040756804, ROUGE-L: 0.23265974808891624
